<a href="https://colab.research.google.com/github/hukim1112/one-day-DL/blob/main/regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

## IMDB 데이터셋 다운로드

이전 노트북에서처럼 임베딩을 사용하지 않고 여기에서는 문장을 멀티-핫 인코딩(multi-hot encoding)으로 변환하겠습니다. 이 모델은 훈련 세트에 빠르게 과대적합될 것입니다. 과대적합을 발생시키기고 어떻게 해결하는지 보이기 위해 선택했습니다.

멀티-핫 인코딩은 정수 시퀀스를 0과 1로 이루어진 벡터로 변환합니다. 정확하게 말하면 시퀀스 `[3, 5]`를 인덱스 3과 5만 1이고 나머지는 모두 0인 10,00 차원 벡터로 변환한다는 의미입니다.

In [ ]:
import numpy as np  # NumPy를 가져와서 다차원 배열 연산을 수행합니다.
from tensorflow import keras  # TensorFlow의 Keras를 가져옵니다.

def multi_hot_sequences(sequences, dimension):
    """
    정수 인덱스 시퀀스를 멀티 핫 인코딩 벡터로 변환합니다.

    Args:
        sequences (list of lists): 각 샘플이 단어 인덱스로 이루어진 리스트.
        dimension (int): 사용할 단어 집합 크기 (단어 인덱스 범위).

    Returns:
        np.ndarray: (num_samples, dimension) 형태의 다차원 배열.
                    해당 단어 인덱스 위치는 1, 나머지는 0.
    """
    # (샘플 개수, 단어 집합 크기) 크기의 0으로 채워진 행렬을 만듭니다.
    results = np.zeros((len(sequences), dimension))

    # 각 샘플(리뷰)의 단어 인덱스를 멀티 핫 벡터로 변환
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # 특정 단어 인덱스에 해당하는 위치를 1로 설정
    return results

# 사용할 단어 집합의 최대 크기 (최상위 1000개의 단어만 유지)
NUM_WORDS = 1000

# IMDb 데이터셋 로드 (num_words=NUM_WORDS: 자주 등장하는 1000개의 단어만 유지)
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

# 데이터셋을 멀티 핫 인코딩 벡터로 변환
train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

# 출력 형태 확인
print("훈련 데이터 형태:", train_data.shape)  # (샘플 수, 1000)
print("테스트 데이터 형태:", test_data.shape)  # (샘플 수, 1000)


만들어진 멀티-핫 벡터 중 하나를 살펴 보죠. 단어 인덱스는 빈도 순으로 정렬되어 있습니다. 그래프에서 볼 수 있듯이 인덱스 0에 가까울수록 1이 많이 등장합니다:

In [ ]:
plt.plot(train_data[0])

이제 학습데이터와 테스트데이터를 토치의 데이터로더로 변환하겠습니다.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 훈련 및 테스트 데이터를 PyTorch Tensor로 변환하여 TensorDataset 생성
train_dataset = TensorDataset(
    torch.tensor(train_data, dtype=torch.float32),  # 입력 데이터 (멀티 핫 벡터)
    torch.tensor(train_labels.reshape(-1, 1), dtype=torch.float32)  # 레이블 (이진 분류)
)

test_dataset = TensorDataset(
    torch.tensor(test_data, dtype=torch.float32),
    torch.tensor(test_labels.reshape(-1, 1), dtype=torch.float32)
)

In [ ]:
#torch 데이터로더를 구축합니다. 배치사이즈는 512개, 학습셋은 shuffe해줍니다.
train_dataloader = DataLoader(dataset=train_dataset, batch_size=512, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=512, shuffle=False)

In [ ]:
# 데이터 로더 샘플 확인
for batch in train_dataloader:
    x_batch, y_batch = batch
    print("입력 데이터 배치 크기:", x_batch.shape)  # (batch_size, NUM_WORDS)
    print("레이블 배치 크기:", y_batch.shape)  # (batch_size, 1)
    break  # 첫 번째 배치만 출력

학습 코드를 작성합니다.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from matplotlib.colors import XKCD_COLORS  # (사용되지 않음 - 필요 없으면 제거 가능)

# 사용할 디바이스 설정 (CUDA 사용 가능 여부 확인)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train_epoch(dataloader, model, loss_fn, optimizer):
    """
    한 에포크(epoch) 동안 모델을 학습하는 함수

    Args:
        dataloader (DataLoader): 훈련 데이터 로더
        model (nn.Module): 학습할 신경망 모델
        loss_fn (torch.nn): 손실 함수 (BCELoss 사용)
        optimizer (torch.optim): 옵티마이저 (Adam, SGD 등)

    Returns:
        tuple: (평균 손실, 정확도)
    """
    size = len(dataloader.dataset)  # 전체 데이터 개수
    num_batches = len(dataloader)  # 미니배치 개수
    model.train()  # 모델을 학습 모드로 설정
    total_loss, correct = 0, 0  # 손실과 정확도 초기화

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # 입력 데이터와 라벨을 GPU/CPU로 이동

        # 모델 예측
        pred = model(X)
        loss = loss_fn(pred, y)  # 손실 계산
        total_loss += loss.item()  # 손실 누적
        correct += ((pred > 0.5) == y).sum().item()  # 정확도 계산 (이진 분류)

        # 역전파 (Backpropagation)
        optimizer.zero_grad()  # 기존 그래디언트 초기화
        loss.backward()  # 그래디언트 계산
        optimizer.step()  # 가중치 업데이트

        # 100 배치마다 손실 출력
        if batch % 100 == 0:
            loss_val, current = loss.item(), batch * len(X)
            print(f"Loss: {loss_val:.6f}  [{current:>5d}/{size:>5d}]")

    # 평균 손실 및 정확도 계산
    total_loss /= num_batches
    correct /= size
    return total_loss, correct  # (평균 손실, 정확도)

def test_epoch(dataloader, model, loss_fn):
    """
    한 에포크(epoch) 동안 모델을 검증하는 함수

    Args:
        dataloader (DataLoader): 검증 데이터 로더
        model (nn.Module): 평가할 신경망 모델
        loss_fn (torch.nn): 손실 함수 (BCELoss 사용)

    Returns:
        tuple: (평균 손실, 정확도)
    """
    size = len(dataloader.dataset)  # 전체 데이터 개수
    num_batches = len(dataloader)  # 미니배치 개수
    model.eval()  # 모델을 평가 모드로 설정
    total_loss, correct = 0, 0  # 손실과 정확도 초기화

    with torch.no_grad():  # 그래디언트 계산 비활성화 (메모리 절약)
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)  # 입력 데이터와 라벨을 GPU/CPU로 이동

            # 모델 예측
            pred = model(X)
            loss = loss_fn(pred, y)  # 손실 계산
            total_loss += loss.item()  # 손실 누적
            correct += ((pred > 0.5) == y).sum().item()  # 정확도 계산

    # 평균 손실 및 정확도 계산
    total_loss /= num_batches
    correct /= size
    return total_loss, correct  # (평균 손실, 정확도)

def train_and_test(model, optimizer):
    """
    모델을 학습하고 검증하는 함수

    Args:
        model (nn.Module): 학습할 신경망 모델
        optimizer (torch.optim): 옵티마이저 (Adam, SGD 등)

    Returns:
        tuple: (훈련 손실 리스트, 검증 손실 리스트)
    """
    epochs = 20  # 총 학습 에포크 수
    train_losses = []  # 훈련 손실 저장 리스트
    val_losses = []  # 검증 손실 저장 리스트
    loss_fn = nn.BCELoss()  # 이진 분류 손실 함수 (Binary Cross Entropy Loss)

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")

        # 훈련 실행
        train_loss, train_acc = train_epoch(train_dataloader, model, loss_fn, optimizer)
        print(f"Train Accuracy: {(100 * train_acc):>0.1f}%, Avg Loss: {train_loss:.6f}")

        # 검증 실행
        val_loss, val_acc = test_epoch(test_dataloader, model, loss_fn)
        print(f"Validation Accuracy: {(100 * val_acc):>0.1f}%, Avg Loss: {val_loss:.6f}")

        # 손실 저장
        train_losses.append(train_loss)
        val_losses.append(val_loss)

    print("Training Complete!")
    return train_losses, val_losses  # 학습 및 검증 손실 반환


## 과대적합(Overfitting)

과대적합은 딥러닝 모델이 훈련 데이터에는 높은 성능을 보이지만, 테스트 데이터에서는 일반화되지 못하는 현상입니다. 이를 방지하는 가장 효과적인 방법은 더 많은 데이터를 확보하는 것입니다. 여의치 않다면, 또 다른 방법은 모델의 규모를 축소하는 것입니다. 즉, 모델에 있는 학습 가능한 파라미터의 수를 줄입니다.

딥러닝에서는 모델의 학습 가능한 파라미터의 수를 종종 모델의 "용량"이라고 말합니다. 직관적으로 생각해 보면 많은 파라미터를 가진 모델이 더 많은 "기억 용량"을 가집니다.

- 모델이 너무 크면 훈련 데이터의 패턴을 암기해버려 테스트 데이터에 일반화되지 않습니다.
- 반대로 모델이 너무 작으면 복잡한 패턴을 학습하지 못해 성능이 떨어집니다.


즉 "너무 많은 용량"과 "충분하지 않은 용량" 사이의 균형을 잡아야 합니다. 안타깝지만 어떤 모델의 (층의 개수나 뉴런 개수에 해당하는) 적절한 크기나 구조를 결정하는 마법같은 공식은 없습니다. 여러 가지 다른 구조를 사용해 실험을 해봐야만 합니다.


### 기준 모델 만들기

In [ ]:
from torch import nn

class BaselineModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fcn = nn.Sequential(
            nn.Linear(1000, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        f = self.fcn(x)
        y = torch.sigmoid(f)
        return y

In [ ]:
model = BaselineModel().to(device)
baseline_train_losses, baseline_test_losses = train_and_test(model, torch.optim.Adam(model.parameters(), lr=1e-3))

### 작은 모델 만들기

앞서 만든 기준 모델과 비교하기 위해 적은 수의 은닉 유닛을 가진 모델을 만들어 보죠:

In [ ]:
class Smaller_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fcn = nn.Sequential(
            nn.Linear(1000, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
        )

    def forward(self, x):
        f = self.fcn(x)
        y = torch.sigmoid(f)
        return y

같은 데이터를 사용해 이 모델을 훈련합니다:

In [ ]:
model = Smaller_model().to(device)
smaller_train_losses, smaller_test_losses = train_and_test(model, torch.optim.Adam(model.parameters(), lr=1e-3))

### 큰 모델 만들기

아주 큰 모델을 만들어 얼마나 빠르게 과대적합이 시작되는지 알아 볼 수 있습니다. 이 문제에 필요한 것보다 훨씬 더 큰 용량을 가진 네트워크를 추가해서 비교해 보죠:

In [ ]:
class Bigger_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fcn = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        f = self.fcn(x)
        y = torch.sigmoid(f)
        return y

역시 같은 데이터를 사용해 모델을 훈련합니다:

In [ ]:
model = Bigger_model().to(device)
bigger_train_losses, bigger_test_losses = train_and_test(model, torch.optim.Adam(model.parameters(), lr=1e-3))

### 훈련 손실과 검증 손실 그래프 그리기

<!--TODO(markdaoust): This should be a one-liner with tensorboard -->

실선은 훈련 손실이고 점선은 검증 손실입니다(낮은 검증 손실이 더 좋은 모델입니다). 여기서는 작은 네트워크가 기준 모델보다 더 늦게 과대적합이 시작되었습니다(즉 에포크 4가 아니라 6에서 시작됩니다). 또한 과대적합이 시작되고 훨씬 천천히 성능이 감소합니다.

In [ ]:
def plot_history(histories, key='binary_crossentropy'):
    plt.figure(figsize=(16,10))

    for name, train_losses, test_losses in histories:
        val = plt.plot(list(range(1,len(test_losses)+1)), test_losses, '--', label=name.title()+' Val')
        plt.plot(list(range(1,len(train_losses)+1)), train_losses, color=val[0].get_color(), label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(list(range(1,len(train_losses)+1)))])
    plt.show()

plot_history([('baseline', baseline_train_losses, baseline_test_losses),
              ('smaller', smaller_train_losses, smaller_test_losses),
              ('bigger', bigger_train_losses, bigger_test_losses)])

큰 네트워크는 거의 바로 첫 번째 에포크 이후에 과대적합이 시작되고 훨씬 더 심각하게 과대적합됩니다. 네트워크의 용량이 많을수록 훈련 세트를 더 빠르게 모델링할 수 있습니다(훈련 손실이 낮아집니다). 하지만 더 쉽게 과대적합됩니다(훈련 손실과 검증 손실 사이에 큰 차이가 발생합니다).

## 과대적합을 방지하기 위한 전략

### 가중치를 규제하기

오캄의 면도날(Occam’s Razor) 원칙에 따르면, 어떤 현상을 설명하는 방법이 여러 가지라면 가장 단순한 설명이 더 정확할 가능성이 높다고 합니다. 신경망 모델도 마찬가지로, 단순한 모델일수록 과대적합 가능성이 낮아집니다.

가중치 규제(Weight Regularization)란?
신경망의 복잡도를 줄이기 위해 가중치가 너무 커지지 않도록 제약을 가하는 방법입니다.
모델이 특정한 패턴을 지나치게 암기하는 것을 방지하고, 가중치의 분포를 균일하게 만들어 일반화를 돕습니다.

가중치 규제 방법
손실 함수에 **큰 가중치에 대한 페널티(추가 비용)**를 포함하는 방식이며, 두 가지 방법이 있습니다.

* [L1 규제](https://developers.google.com/machine-learning/glossary/#L1_regularization)는 가중치의 절댓값에 비례하는 비용이 추가됩니다(즉, 가중치의 "L1 노름(norm)"을 추가합니다). 일부 가중치를 0으로 만들어 희소한 모델을 생성합니다.

* [L2 규제](https://developers.google.com/machine-learning/glossary/#L2_regularization)는 가중치의 제곱에 비례하는 비용이 추가됩니다(즉, 가중치의 "L2 노름"의 제곱을 추가합니다). 가중치를 0에 가깝게 하지만, 완전히 0으로 만들지는 않습니다. L2 규제는 "가중치 감쇠(weight decay)"라고도 불리며, 두 개념은 같은 의미입니다.

일반적으로 L2 규제가 더 자주 사용됩니다.

In [ ]:
model = Bigger_model().to(device)
l2_train_losses, l2_test_losses = train_and_test(model, torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.005))

```l2(0.001)```는 네트워크의 전체 손실에 층에 있는 가중치 행렬의 모든 값이 ```0.001 * weight_coefficient_value**2```만큼 더해진다는 의미입니다. 이런 페널티(penalty)는 훈련할 때만 추가됩니다. 따라서 테스트 단계보다 훈련 단계에서 네트워크 손실이 훨씬 더 클 것입니다.

L2 규제의 효과를 확인해 보죠:

In [ ]:
plot_history([('biggermodel', bigger_train_losses, bigger_test_losses),
              ('l2', l2_train_losses, l2_test_losses)])

결과에서 보듯이 모델 파라미터의 개수는 같지만 L2 규제를 적용한 모델이 기본 모델보다 과대적합에 훨씬 잘 견디고 있습니다.

### 드롭아웃 추가하기

드롭아웃(dropout)은 신경망에서 가장 효과적이고 널리 사용하는 규제 기법 중 하나입니다. 토론토(Toronto) 대학의 힌튼(Hinton)과 그의 제자들이 개발했습니다. 드롭아웃을 층에 적용하면 훈련하는 동안 층의 출력 특성을 랜덤하게 끕니다(즉, 0으로 만듭니다). 훈련하는 동안 어떤 입력 샘플에 대해 [0.2, 0.5, 1.3, 0.8, 1.1] 벡터를 출력하는 층이 있다고 가정해 보죠. 드롭아웃을 적용하면 이 벡터에서 몇 개의 원소가 랜덤하게 0이 됩니다. 예를 들면, [0, 0.5, 1.3, 0, 1.1]가 됩니다. "드롭아웃 비율"은 0이 되는 특성의 비율입니다. 보통 0.2에서 0.5 사이를 사용합니다. 테스트 단계에서는 어떤 유닛도 드롭아웃하지 않습니다. 훈련 단계보다 더 많은 유닛이 활성화되기 때문에 균형을 맞추기 위해 층의 출력 값을 드롭아웃 비율만큼 줄입니다.

네트워크에 두 개의 `Dropout` 층을 추가하여 과대적합이 얼마나 감소하는지 알아 보겠습니다:

In [ ]:
class Dropout_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fcn = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.75),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.75),
            nn.Linear(512, 1),
        )
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        f = self.fcn(x)
        y = torch.sigmoid(f)
        return y


In [ ]:
model = Dropout_model().to(device)
dpt_train_losses, dpt_test_losses = train_and_test(model, torch.optim.Adam(model.parameters(), lr=1e-3))

In [ ]:
plot_history([('biggermodel', bigger_train_losses, bigger_test_losses),
              ('dropout', dpt_train_losses, dpt_test_losses)])

드롭아웃을 추가하니 기준 모델보다 확실히 향상되었습니다.

정리하면 신경망에서 과대적합을 방지하기 위해 가장 널리 사용하는 방법은 다음과 같습니다:

* 더 많은 훈련 데이터를 모읍니다.
* 네트워크의 용량을 줄입니다.
* 가중치 규제를 추가합니다.
* 드롭아웃을 추가합니다.